In [10]:
import boto3
import time
import urllib
import json

In [11]:
# Import our AWS credentials
import os
AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_SECRET_ACCESS_KEY']

In [22]:
# AWS job info
job_name = 'lectures-753d'
job_uri = 's3://lectures-753d/Feynman_Cassette_1.mp3'

In [23]:
transcribe = boto3.client('transcribe', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY, region_name='us-west-2')

In [24]:
# Run transcribe job
transcribe.start_transcription_job(TranscriptionJobName=job_name, Media={'MediaFileUri': job_uri}, MediaFormat='mp3', LanguageCode='en-US')


{'TranscriptionJob': {'TranscriptionJobName': 'lectures-753d',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'en-US',
  'MediaFormat': 'mp3',
  'Media': {'MediaFileUri': 's3://lectures-753d/Feynman_Cassette_1.mp3'},
  'StartTime': datetime.datetime(2021, 4, 12, 22, 22, 35, 351000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2021, 4, 12, 22, 22, 35, 319000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': 'f282642e-2964-4e44-8bee-4013db774dfc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 13 Apr 2021 05:22:34 GMT',
   'x-amzn-requestid': 'f282642e-2964-4e44-8bee-4013db774dfc',
   'content-length': '273',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [31]:
status = transcribe.list_transcription_jobs()
status

{'TranscriptionJobSummaries': [{'TranscriptionJobName': 'lectures-753d',
   'CreationTime': datetime.datetime(2021, 4, 12, 22, 22, 35, 319000, tzinfo=tzlocal()),
   'StartTime': datetime.datetime(2021, 4, 12, 22, 22, 35, 351000, tzinfo=tzlocal()),
   'CompletionTime': datetime.datetime(2021, 4, 12, 22, 32, 31, 732000, tzinfo=tzlocal()),
   'LanguageCode': 'en-US',
   'TranscriptionJobStatus': 'COMPLETED',
   'OutputLocationType': 'SERVICE_BUCKET'}],
 'ResponseMetadata': {'RequestId': '37bc2bd2-469e-4adc-b42d-da5a36a3fef2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 13 Apr 2021 05:32:38 GMT',
   'x-amzn-requestid': '37bc2bd2-469e-4adc-b42d-da5a36a3fef2',
   'content-length': '265',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [33]:
# Check job status
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(15)
print(status)

{'TranscriptionJob': {'TranscriptionJobName': 'lectures-753d', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'en-US', 'MediaSampleRateHertz': 44100, 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 's3://lectures-753d/Feynman_Cassette_1.mp3'}, 'Transcript': {'TranscriptFileUri': 'https://s3.us-west-2.amazonaws.com/aws-transcribe-us-west-2-prod/981176631646/lectures-753d/a4cf6952-1b65-4799-947a-1bb93b7df8ba/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEHUaCXVzLXdlc3QtMiJHMEUCIE96FW%2Fzquyyc2YDZObrscG3gI3BFD937ZBwmOz6%2FkXpAiEAoS5YJ3hdY5JijgWkMwHU2kgqDZy20zyIckSJpZxnNQoqvQMIzv%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARACGgwwODAyNDgzMjIyMDYiDGSWSbSJGlFuZmXJ5yqRA5xH0hqSQJZctytLtwcXH4qI8NWwYeL97unm%2FFFv4oBO%2Fzb6mp26m%2B3X0dTPaLqi%2FKmGCv75SKKn6bq0l%2FXeovITtstMPfB1P2c4bwAit7WvSZwIyNYNzjjLQVvlluxa0krGYYVCIktNjXpAQfwPyop4U%2BPl6eRN8SUDwKmlqXVDZ5iWWfgP12rNhunEk58Kwtt8FrUQ9jiZRScqyvvHHzNcvfmsv%2BFhifm3mZMcVTJ1I7tOzDziKa1%2Bqt%2F%2BYL4j3Hc3u1navsfVDdg6fI2NAwcALzRk56CubfSJ8UjN1Tdzvs9SHmIhjf

In [36]:
# Retrieve text
if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    response = urllib.request.urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = json.loads(response.read())
    text = data['results']['transcripts'][0]['transcript']
    
    with open('results.txt', 'w+') as f:
        f.write(text)